In [45]:
import docx2txt
import pandas as pd

### setup
filePath = 'engBook_vocabulary.docx'
text = docx2txt.process(filePath)
texts = []
introduction_text = ""

amount_texts = 167
amount_questions = 10
tmpABC = ["A)", "B)", "C)", "D)", "E)"]
amount_choices = len(tmpABC)

#split document into text+question+answer blocks (50 texts-> 50 blocks)
for i in range(0,amount_texts, amount_questions):
    tmp=text.split("Question " + str(i+1), 1)
    tmp[0]=tmp[0].replace('\n', ' ').replace('\r', '')
    texts.append("Question " + str(i-amount_questions+1)  + tmp[0])
    #print("Question " + str(i-amount_questions+1) + " " + tmp[0])
    if(len(tmp) > 1):
        text=tmp[1]
    
introduction_text=texts.pop(0).replace("Text 0 ", "")

print(texts)


['Question 1: Which of the following statements about English vocabulary is/are correct?  A) "To be meticulous" means to be extremely careful about small details. (True/False) True   B) "Innovative" and "outdated" are synonyms. (True/False) False   C) "To exacerbate" a problem means to make it better. (True/False) False   D) "A paradigm shift" refers to a significant change in the way of thinking or doing something. (True/False) True   E) "Ambiguous" means that something is very clear and precise. (True/False) False    Question 2: Which of the following statements about vocabulary is/are correct?  A) "Subsequent" means happening before something. (True/False) False   B) "To scrutinize" means to examine or inspect closely and thoroughly. (True/False) True   C) "An anecdote" is a detailed and comprehensive analysis of a subject. (True/False) False   D) "Ubiquitous" means found everywhere; prevalent. (True/False) True   E) "Ostentatious" refers to something done in a modest and unassuming

In [46]:
#split each block into QuestionType, QuestionText, Choice1 - Choice7
#choice which is "true" is marked with a "*" before the text

#### setup
tmpTexts=[]
tmpTexts.append(texts[0])
tmpTexts.append(texts[1])

for main_index,block in enumerate(tmpTexts):
    # setup
    choices=[]
    question_text = ""
    questions = []
    excel_rows = []
    dataFrame_columns = ["QuestionType", "QuestionText", "Explanation", "Choice1", "Choice2", "Choice3", "Choice4", "Choice5", "Choice6", "Choice7", "Choice8", "Choice9", "Choice10"]
    writer = pd.ExcelWriter( "Vocabulary " + str(main_index+1) + ".xlsx", engine="xlsxwriter")
    tmpQuestions=block
    #print(main_index)
    #print(tmpQuestions)
    
    # Questions Blöcke aufteilen (question + choices)
    for i in range(amount_questions):
        if(i == amount_questions-1):
            tmp = tmpQuestions.split("Question " + str(main_index*10 + i+2), 1)
            #print(tmp)
            questions.append(tmp[0])
            #questions.append("Question " + str(main_index*10 + i+2) + tmp[1])
            
        else:
            tmp = tmpQuestions.split("Question " + str(main_index*10+i+2), 1)
            #print(tmp)
            questions.append(tmp[0])
            tmpQuestions = "Question " + str(main_index*10+i+2) + tmp[1]
    #print(questions)  
    # choices aufteilen & true/false bestimmen
    for index,question in enumerate(questions):
        choices=[]
        explanation="<strong>Correct answers: </strong>"
        tmp = question.split(tmpABC[0], 1)
        question_text = "<strong>" + tmp[0].replace("Question " + str(main_index*10+index+1)+ ":", "") + "</strong>"
        tmpChoices=tmpABC[0] + tmp[1]
        for index, letter in enumerate(tmpABC):
            if(letter == tmpABC[0]):
                continue
            elif(letter == tmpABC[-1]):
                tmp = tmpChoices.split("  " + letter, 1)
                #print(tmp)
                choices.append(tmp[0].replace(tmpABC[-2] + " ",""))
                choices.append(tmp[1].replace(" ", "", 1))
            else:
                tmp = tmpChoices.split("  " + letter, 1)
                #print(tmp)
                choices.append(tmp[0].replace(tmpABC[index-1] + " ", ""))
                tmpChoices=letter + tmp[1]
        #print(choices)
        for index,choice in enumerate(choices):
            tmp=choice.split(" (True/False) ")
            if(tmp[1].replace(" ", "") == "True"):
                choices[index] = "*" + tmp[0]
                explanation = explanation + "<br>" + '<span style="color: rgb(97, 189, 109);">' +  tmpABC[index] + " " + tmp[0] + '</span>'+ " "
            else:
                choices[index] = tmp[0]
                explanation = explanation + "<br>" + '<span style="color: rgb(226, 80, 65);">' +  tmpABC[index] + " " + tmp[0] + '</span>'+ " "
        
        ### pandas setup (writing to excel table)
        values=["MA", question_text, explanation] + choices + ["","","","",""]
        excel_rows.append(values)

    df = pd.DataFrame ( excel_rows, columns = dataFrame_columns)
    df.to_excel(writer, sheet_name="QUESTIONS", startrow=0, header=True, index=False)
    writer.close()          
    # print(question_text)
    # print(tmpQuestionText)
    # print(choices)
    # print(questions)